In [ ]:
import pandas as pd
import os
import openai
import praw # Python Reddit Wrapper

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
client_id = "LCwio15ckIOGm9xSXg5AEQ"
client_secret = "5YsjsV7HP2qdbeAKslCyCxMpCYqCIQ"

In [ ]:
# Create the connection to Reddit
reddit = praw.Reddit(client_id=client_id,
                    client_secret=client_secret,
                    user_agent="sentiment_analysis")

In [ ]:
# With this reddit connection, tt is possible to have posts and comments in a subreddit to scrap for our sentiment analysis.
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit = 5, num_comments = 2, skip_first =2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for post_counter, post in enumerate(getattr(subreddit, sub_instance)(limit = limit)):
        if post_counter < skip_first:
            continue
        post_counter+= (1-skip_first)
        
        titles_and_comments[post_counter] = ""

        submission = reddit.submission(post.id)
        title = post.title

        titles_and_comments[post_counter] += "Title: " + title  + "\n\n"
        titles_and_comments[post_counter] += "Comments: \n\n"
        
        comment_counter = 0

        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[post_counter] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments


In [ ]:
titles_and_comments = get_titles_and_comments()

In [ ]:
titles_and_comments

In [ ]:
print(titles_and_comments[1])

In [ ]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments

In [ ]:
prompt = create_prompt(titles_and_comments[1])
print(prompt)

In [ ]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)